In [1]:
import numpy as np
import math
import random

class Tetromino():

    TYPES = [' ', 'I', 'O', 'T', 'S', 'Z', 'J', 'L']
    TYPES_D = {
        ' ': 0,
        'I': 1,
        'O': 2,
        'T': 3,
        'S': 4,
        'Z': 5,
        'J': 6,
        'L': 7
    }

    def __init__(self, state):
        self.state = np.array(state, dtype=np.uint8, copy=True)

    @staticmethod
    def ITetromino():
	    return Tetromino(
            [
                [1, 1, 1, 1]
            ]
        )

    @staticmethod
    def OTetromino():
        return Tetromino(
            [
                [2, 2],
                [2, 2]
            ]
        )

    @staticmethod
    def TTetromino():
        return Tetromino(
            [
                [3, 3, 3],
                [0, 3, 0]
            ]
        )

    @staticmethod
    def STetromino():
        return Tetromino(
            [
                [0, 4, 4],
                [4, 4, 0]
            ]
        )

    @staticmethod
    def ZTetromino():
        return Tetromino(
            [
                [5, 5, 0],
                [0, 5, 5]
            ]
        )

    @staticmethod
    def JTetromino():
        return Tetromino(
            [
                [6, 6, 6],
                [0, 0, 6]
            ]
        )

    @staticmethod
    def LTetromino():
        return Tetromino(
            [
                [7, 7, 7],
                [7, 0, 0]
            ]
        )

    @staticmethod
    def create(letter):
        if letter.upper() in Tetromino.TYPES[1:]:
            raise ValueError('No Tetromino of type {}'.format(letter))
        return getattr(Tetromino, '{}Tetromino'.format(letter.upper()))()

    def __str__(self):
        return str(np.vectorize(Tetromino.TYPES.__getitem__)(self.state))

    def __getitem__(self, key):
        return self.state[key]

    def copy(self):
        return Tetromino(self.state)

    def flat(self):
        return self.state.flat

    def width(self):
        return self.state.shape[1]

    def height(self):
        return self.state.shape[0]

    def rotate(self, change):
        while change < 0:
            change += 4
        change = (change % 4)
        if change == 0:
            return None
        elif change == 1:
            self.rotate_right()
        elif change == 2:
            self.flip()
        elif change == 3:
            self.rotate_left()

    def rotate_right(self):
        self.state = np.rot90(self.state, 3)
        return self

    def rotate_left(self):
        self.state = np.rot90(self.state, 1)
        return self

    def flip(self):
        self.state = np.rot90(self.state, 2)
        return self
class Field():

    WIDTH = 10
    HEIGHT = 20
    SCORING_ELEMENTS = 6

    def __init__(self, state=None):
        """
        Initializes a Tetris Field.
        Rows increase downward and columns increase to the right.
        """
        if state is not None:
            self.state = np.array(state, dtype=np.uint8, copy=True)
        else:
            self.state = np.full((Field.HEIGHT, Field.WIDTH), 0, dtype=np.uint8)

    def __str__(self):
        """
        Returns a string representation of the field.
        """
        bar = '   |' + ' '.join(map(str, range(Field.WIDTH))) + '|\n'
        mapped_field = np.vectorize(Tetromino.TYPES.__getitem__)(self.state)
        field = '\n'.join(['{:2d} |'.format(i) +
            ' '.join(row) + '|' for i, row in enumerate(mapped_field)])
        return bar + field + '\n' + bar

    def _test_tetromino_(self, tetromino, r_start, c_start):
        """
        Tests to see if a tetromino can be placed at the specified row and
        column. It performs the test with the top left corner of the
        tetromino at the specified row and column.
        """
        r_end, c_end = r_start + tetromino.height(), c_start + tetromino.width()
        if c_start < 0 or c_end > Field.WIDTH:
            return False
        if r_start < 0 or r_end > Field.HEIGHT:
            return False
        test_area = self.state[r_start:r_end, c_start:c_end]
        for s, t in zip(test_area.flat, tetromino.flat()):
            if s != 0 and t != 0:
                return False
        return True

    def _place_tetromino_(self, tetromino, r_start, c_start):
        """
        Place a tetromino at the specified row and column.
        The bottom left corner of the tetromino will be placed at the specified
        row and column. This function does not perform checks and will overwrite
        filled spaces in the field.
        """
        r_end, c_end = r_start + tetromino.height(), c_start + tetromino.width()
        if c_start < 0 or c_end > Field.WIDTH:
            return False
        if r_start < 0 or r_end > Field.HEIGHT:
            return False
        for tr, sr in enumerate(range(r_start, r_end)):
            for tc, sc, in enumerate(range(c_start, c_end)):
                if tetromino[tr][tc] != 0:
                    self.state[sr][sc] = tetromino[tr][tc]

    def _get_tetromino_drop_row_(self, tetromino, column):
        """
        Given a tetromino and a column, return the row that the tetromino
        would end up in if it were dropped in that column.
        Assumes the leftmost column of the tetromino will be aligned with the
        specified column.
        """
        if column < 0 or column + tetromino.width() > Field.WIDTH:
            return -1
        last_fit = -1
        for row in range(tetromino.height(), Field.HEIGHT):
            if self._test_tetromino_(tetromino, row, column):
                last_fit = row
            else:
                return last_fit
        return last_fit
    
    def copy(self):
        """
        Returns a shallow copy of the field.
        """
        return Field(self.state)

    def _line_clear_(self):
        """
        Checks and removes all filled lines.
        """
        filled = np.array([row.all() and row.any() for row in self.state])
        num_clear_lines = len(self.state[filled])
        non_filled = np.array(
            [not row.all() and row.any() for row in self.state])
        if non_filled.any():
            tmp = self.state[non_filled]
            self.state.fill(0)
            self.state[Field.HEIGHT - tmp.shape[0]:] = tmp
        return num_clear_lines

    def drop(self, tetromino, column):
        """
        Drops a tetromino in the specified column.
        The leftmost column of the tetromino will be aligned with the specified
        column.
        Returns the row it was dropped in for computations or -1 if a drop was
        unable to be computed.
        """
        assert isinstance(tetromino, Tetromino)
        row = self._get_tetromino_drop_row_(tetromino, column)
        if row == -1:
            return row, -1
        self._place_tetromino_(tetromino, row, column)
        num_lines_cleared = self._line_clear_()
        return row, num_lines_cleared

    def count_gaps(self):
        """
        Check each column one by one to make sure there are no gpas in the
        column.
        """
        rows_with_holes = set()
        well_cells = 0
        gaps = 0
        filled_rows_above_highest_hole = 0
        # Cut off all the empty space above all the placed tetrominos
        top_indices = np.argmax(self.state.T != 0, axis = 1)
        # Count the number of gaps past the first filled space per column
        for col, top in zip(self.state.T, top_indices):
            for index in range(top, 20):
                if col[index] == 0:
                    gaps += 1
                    rows_with_holes.add(index)
        lst = sorted(rows_with_holes)
        if len(rows_with_holes):
            highest_hole = 20 - lst[0]
        else:
            highest_hole = 0

        for pos, row in enumerate(self.state):
            if pos == highest_hole:
                break
            elif row.all():
                filled_rows_above_highest_hole += 1

        top_indices -= 1
        for col in range(1,9):
            cur_row = top_indices[col]
            if not self.state[cur_row][col] and self.state[cur_row][col - 1] and self.state[cur_row][col + 1]:
                well_cells += 1
        return len(rows_with_holes), gaps, highest_hole, filled_rows_above_highest_hole, well_cells

    def heights(self):
        """
        Return an array containing the heights of each column.
        """
        heights = Field.HEIGHT - np.argmax(self.state.T != 0, axis=1)
        max_height = np.amax(heights)
        total_bumpiness = 0
        max_bumpiness = 0
        for i in range(9):
            bumpiness = abs(heights[i]- heights[i+1])
            max_bumpiness = max(bumpiness, max_bumpiness)
            total_bumpiness += bumpiness
        return max_height, max_bumpiness, total_bumpiness

    def get_scoring_vector(self):
        """
        Get a vector of values derived from the field used to score a tetromino
        placement.
        """
        max_height, max_bumpiness, total_bumpiness = self.heights()
        rows_with_holes, holes, highest_hole, filled_rows_above_highest_hole, well_cells = self.count_gaps()
        return np.array([
            holes,                          # number of holes
            total_bumpiness,                # sum of bumpiness
            max_height,                     # maximum height
            np.count_nonzero(self.state),   # number of blocks
            rows_with_holes,                # rows with holes
            max_bumpiness,                  # max bumpiness
            well_cells,                     # well cells
            highest_hole,                   # highest hole
            filled_rows_above_highest_hole  # rows above the highest hole
        ])

    def get_optimal_drop(self, tetromino, weights=None):
        """
        Given a tetromino and a vector of scoring weights, this method
        calculates the best placement of the tetromino, scoring each placement
        with the weight vector.
        """
        rotations = [
            tetromino,
            tetromino.copy().rotate_right(),
            tetromino.copy().flip(),
            tetromino.copy().rotate_left()
        ]

        best_field = None
        best_drop_score = math.inf
        for rotation, tetromino_ in enumerate(rotations):
            for column in range(Field.WIDTH):
                f = self.copy()
                row, num_lines_cleared = f.drop(tetromino_, column)
                if row == -1:
                    continue
                scoring_vector = f.get_scoring_vector()
                scoring_vector = np.insert(scoring_vector, 0, num_lines_cleared)
                if weights is not None:
                    score = scoring_vector.dot(weights)
                else:
                    score = scoring_vector.sum()
                if score < best_drop_score:
                    best_drop_score = score
                    best_field = f
        return best_field, num_lines_cleared
GENES = 10

class Chromosome():

    N_SIMULATIONS = 10
    MUTATION_CHANCE = 0.01

    @staticmethod
    def random():
        """
        Returns a random initialized Chromosome.
        """
        return Chromosome(Chromosome.random_genes())

    @staticmethod
    def random_genes():
        """
        Returns a randomly initialized array for the genetic values inside
        a Chromosome. The genetic values are sampled from the open interval from
        -1 to 1.
        """
        return (np.random.random_sample(GENES) * 2) - 1

    @staticmethod
    def set_globals(n_simulations,mutation_chance):
        """
        Sets the static constants that affect how chromosomes will cross and
        be evaluated.
        """
        Chromosome.N_SIMULATIONS = n_simulations
        Chromosome.MUTATION_CHANCE = mutation_chance

    def __init__(self, genes=None):
        """
        Initializes a Chromosome.
        """
        self.genes = genes
        self.fitness = None
        self.field_score = None
        self.length = None
        self.clear = None
        self.simulations = Chromosome.N_SIMULATIONS
        self.mutation_chance = Chromosome.MUTATION_CHANCE

    def __str__(self):
        """
        Returns a string representation of the chromosome's genes.
        """
        return 'Weights{}\n# of blocks dropped{}\n# of lines cleared{}\n Sccore{}'.format(self.genes, self.length, self.clear, self.fitness)

    def cross(self, other):
        """
        Performs genetic crossing between this chromosome and another
        chromosome, returning a new chromosome.
        The genetic values stored in a chromosome are all numeric in the open
        interval from -1 to 1. When we cross two chromosomes, we will take the
        weighted average between the genetic values of the two chromosomes,
        weighting it according to the fitness value of each chromosome.
        During chromosomal crossing, a mutation has a chance to occur for each
        genetic value. If one occurs, that genetic value will be set to a new
        random number in the open interval from -1 to 1.
        """
        w_sum = self.get_fitness() + other.get_fitness()
        genes = (self.genes * self.get_fitness() / w_sum) + (
            other.genes * other.fitness / w_sum)
        # Each genetic value has a chance to mutate.
        mutated_genes = np.random.random_sample(GENES) < self.mutation_chance
        if np.any(mutated_genes):
            nonmutated_genes = mutated_genes == 0
            mutation = Chromosome.random_genes()
            # Compose the original genetic values with the mutated genetic
            # values.
            genes = (genes * nonmutated_genes) + (mutation * mutated_genes)
        return Chromosome(genes)

    def get_fitness(self):
        """
        Returns the fitness of the chromosome. If it is cached, it will fetch
        the cached fitness value, otherwise it will compute it and return it.
        """
        if self.fitness is not None:
            return self.fitness
        self.recalculate_fitness()
        return self.fitness

    def recalculate_fitness(self):
        """
        Calculates the fitness values of the chromosome by running a number
        of Tetris simulations using the chromosome and averaging their
        performance.
        """
        scores = np.array([
            self._get_fitness_() for _ in range(self.simulations)])
        self.length, self.clear = np.mean(scores, axis=0)
        self.fitness = self.length + (10 * self.clear * self.clear)

    def _get_fitness_(self):
        """
        Helper method to perform a single simulation to evaluate the performance
        of the chromosome. This will be called multiple times and the overall
        performance of the chromosome is the average of all the runs.
        """
        tetrominos = [
            Tetromino.ITetromino(),
            Tetromino.OTetromino(),
            Tetromino.TTetromino(),
            Tetromino.STetromino(),
            Tetromino.ZTetromino(),
            Tetromino.JTetromino(),
            Tetromino.LTetromino()
        ]
        field = Field()
        fitness = 0
        length = 0
        clear = 0 
        while True:
            tetromino = random.choice(tetrominos)
            _field, cleared_lines = field.get_optimal_drop( tetromino, self.genes)
            if _field is None :
                break
            else:
                length += 1
                clear += cleared_lines
                field = _field
                # fitness = length + 10 * clear * clear
        # fitness = length + (10 * clear * clear)
        return length, clear
class Generation():

    def __init__(self, population, selection, threshold):
        """
        Initializes a Generation of individuals/chromosomes.
        """
        self.population = population
        self.threshold = threshold
        self.generations = 0
        self.selection = selection

    def run(self):
        """
        This method will run the genetic algorithm on the population for the
        given number of generations.
        """
        last = None
        
        for i in range(20):
            # Sort the population of chromosomes by their fitness
            population_by_fitness = sorted(
                self.population, key=lambda gene: gene.get_fitness())
            fittest_member = population_by_fitness[-1]
            # if last is not None and fittest_member.get_fitness() - last.get_fitness() <= self.threshold:
            #     return last
            # else:
            #     last = fittest_member
            print('Generation: {}'.format(self.generations))
            print([member.get_fitness() for member in population_by_fitness])
            print('Fittest member: {}'.format(fittest_member))
            print('')
            # calculate how many members should be removed
            length = len(population_by_fitness)
            cut = math.floor(length * self.selection) 
            # Memebers in the population after selection
            fittest = population_by_fitness[cut:]
            # Shuffle and cross breed the fittest members.
            random.shuffle(fittest)
            # new_gen = []
            for i in range(0, length - cut, 2):
                if i+1 < length - cut:
                    fittest.append(fittest[i].cross(fittest[i + 1]))
                # else:
                #     new_gen.append(fittest[i])
            self.population = fittest
            for chromosome in self.population:
                chromosome.recalculate_fitness()
            self.generations += 1
        return sorted(self.population, key=lambda gene: gene.get_fitness())[-1]

In [ ]:
Chromosome.set_globals(10,0.1) # initialize macro for N and q
population = Generation([Chromosome.random() for i in range(25)], 0.4, 0) 
# initialize macro for p and stopping_criteria
fittest = population.run()
print('Fittest member: {}'.format(fittest))